In [22]:
from glob import glob
import os

root = './datasets/tom_and_jerry/original'

directories = glob(os.path.join(root, '*'))
directory_names = []
for directory in directories:
    directory_names.append(directory[directory.rindex('\\') + 1:])

print(directory_names)

['jerry', 'tom', 'tom_jerry_0', 'tom_jerry_1']


In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
root = './datasets/tom_and_jerry/original'
# root = './datasets/face/test/'
image_data_generator = ImageDataGenerator(rescale=1./255)

generator = image_data_generator.flow_from_directory(root, target_size=(720, 720), batch_size=32, class_mode='categorical')
print(generator.class_indices)
# generator_test = image_data_generator.flow_from_directory(root, target_size=(48, 48), batch_size=32, class_mode='categorical')
# print(generator_test.class_indices)

Found 5478 images belonging to 4 classes.
{'jerry': 0, 'tom': 1, 'tom_jerry_0': 2, 'tom_jerry_1': 3}


In [24]:
import pandas as pd

tj_df = pd.DataFrame({'file_paths': generator.filepaths, 'targets': generator.classes})
tj_df

,file_paths,targets
0,./datasets/tom_and_jerry/original\jerry\frame1...,0
1,./datasets/tom_and_jerry/original\jerry\frame1...,0
2,./datasets/tom_and_jerry/original\jerry\frame1...,0
3,./datasets/tom_and_jerry/original\jerry\frame1...,0
4,./datasets/tom_and_jerry/original\jerry\frame1...,0
...,...,...
5473,./datasets/tom_and_jerry/original\tom_jerry_1\...,3
5474,./datasets/tom_and_jerry/original\tom_jerry_1\...,3
5475,./datasets/tom_and_jerry/original\tom_jerry_1\...,3
5476,./datasets/tom_and_jerry/original\tom_jerry_1\...,3


In [25]:
tj_df.loc[:, 'file_paths'] = tj_df.file_paths.apply(lambda x: x.replace('\\', '/'))
tj_df

,file_paths,targets
0,./datasets/tom_and_jerry/original/jerry/frame1...,0
1,./datasets/tom_and_jerry/original/jerry/frame1...,0
2,./datasets/tom_and_jerry/original/jerry/frame1...,0
3,./datasets/tom_and_jerry/original/jerry/frame1...,0
4,./datasets/tom_and_jerry/original/jerry/frame1...,0
...,...,...
5473,./datasets/tom_and_jerry/original/tom_jerry_1/...,3
5474,./datasets/tom_and_jerry/original/tom_jerry_1/...,3
5475,./datasets/tom_and_jerry/original/tom_jerry_1/...,3
5476,./datasets/tom_and_jerry/original/tom_jerry_1/...,3


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(tj_df.file_paths, tj_df.targets, stratify=tj_df.targets, test_size=0.2, random_state=124)

print(y_train.value_counts())
print(y_test.value_counts())

targets
1    1544
2    1222
0     992
3     624
Name: count, dtype: int64
targets
1    386
2    306
0    248
3    156
Name: count, dtype: int64


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=124)

print(y_train.value_counts())
print(y_val.value_counts())

targets
1    1235
2     977
0     794
3     499
Name: count, dtype: int64
targets
1    309
2    245
0    198
3    125
Name: count, dtype: int64


In [28]:
import shutil

root = './datasets/tom_and_jerry/'

for file_path in X_train:
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    destination = os.path.join(root, 'train/' + animal_dir)

    if not os.path.exists(destination):
        os.makedirs(destination)

    shutil.copy2(file_path, destination)

In [29]:
import shutil

root = './datasets/tom_and_jerry/'

for file_path in X_val:
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    destination = os.path.join(root, 'validation/' + animal_dir)

    if not os.path.exists(destination):
        os.makedirs(destination)

    shutil.copy2(file_path, destination)

In [30]:
import shutil

root = './datasets/tom_and_jerry/'

for file_path in X_test:
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    destination = os.path.join(root, 'test/' + animal_dir)

    if not os.path.exists(destination):
        os.makedirs(destination)

    shutil.copy2(file_path, destination)

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 720

train_dir = './datasets/tom_and_jerry/train'
validation_dir = './datasets/tom_and_jerry/validation'
test_dir = './datasets/tom_and_jerry/test'

train_data_generator = ImageDataGenerator(rescale=1./255)
validation_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

train_generator = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=8,
    class_mode='categorical'
)

validation_generator = validation_data_generator.flow_from_directory(
    validation_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=8,
    class_mode='categorical'
)

test_generator = test_data_generator.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=8,
    class_mode='categorical'
)

print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

Found 3505 images belonging to 4 classes.
Found 877 images belonging to 4 classes.
Found 1096 images belonging to 4 classes.
{'jerry': 0, 'tom': 1, 'tom_jerry_0': 2, 'tom_jerry_1': 3}
{'jerry': 0, 'tom': 1, 'tom_jerry_0': 2, 'tom_jerry_1': 3}
{'jerry': 0, 'tom': 1, 'tom_jerry_0': 2, 'tom_jerry_1': 3}


In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2

IMAGE_SIZE = 720

input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# alpha를 크게 할 수록 Weight값을 작게 만들어서 과적합을 개선할 수 있고
# alpha를 작게 할 수록 Weight의 값이 커지지만, 어느 정도 상쇄하므로 과소적합을 개선할 수 있다.
x = Conv2D(filters=256, kernel_size=3, padding='same', kernel_regularizer=l2(1e-5), kernel_initializer='he_normal')(input_tensor)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=256, kernel_size=3, padding='same', kernel_regularizer=l2(1e-5), kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(2)(x)

x = Conv2D(filters=512, kernel_size=3, padding='same', kernel_regularizer=l2(1e-5), kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(filters=512, kernel_size=3, padding='same', kernel_regularizer=l2(1e-5), kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(2)(x)

x = GlobalAveragePooling2D()(x)
x = Dropout(rate=0.5)(x)
x = Dense(300, activation='relu', kernel_regularizer=l2(1e-5), kernel_initializer='he_normal')(x)
x = Dropout(rate=0.3)(x)
output = Dense(4, activation='softmax', kernel_initializer='glorot_normal')(x)

model = Model(inputs=input_tensor, outputs=output)

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 720, 720, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 720, 720, 256)       │           7,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 720, 720, 256)       │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 720, 720, 256)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 720, 720, 256)       │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 720, 720, 256)       │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 720, 720, 256)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 360, 360, 256)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 360, 360, 512)       │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 360, 360, 512)       │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_6 (Activation)            │ (None, 360, 360, 512)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 360, 360, 512)       │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 360, 360, 512)       │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_7 (Activation)            │ (None, 360, 360, 512)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 180, 180, 512)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 300)                 │         153,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 300)                 │              

 Total params: 4,298,464 (16.40 MB)

 Trainable params: 4,295,392 (16.39 MB)

 Non-trainable params: 3,072 (12.00 KB)

In [34]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])


In [35]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

In [ ]:
history = model.fit(
    train_generator,
    batch_size=32,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[mcp_cb, rlr_cb, ely_cb]
)

Epoch 1/20


C:\Users\HANSUNG\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
